In [1]:
import geopandas as gpd
import shapely
import shapely
import rasterio
import glob
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append("..")
import os
import datetime
import profiling_tools
import altair as alt
from altair import datum
import json
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Inputs
Provide:
- Input file path to file with cross section lines/polygons to extract low points/stream profile from
- Output file path where low points will be saved
- Input directory path to location of DEMs
- Parameter `LINE_COMPLEXITY` which is the number of points that each cross-section line is split into. `LINE_COMPLEXITY` elevation points will be extracted from the DEM for each cross section line

If you use the arg, you must run from CLI like this

```
HSFM_GEOMORPH_INPUT_FILE='inputs/mazama_inputs.json' jupyter nbconvert --execute --to html dem-analysis/mt_baker_mass_wasted/transects.ipynb  --output outputs/transects_mazama.html
```

In [2]:
# json_file_path = 'inputs/mazama_inputs.json'
# Or set an env arg:
json_file_path = os.environ['HSFM_GEOMORPH_INPUT_FILE']

In [3]:
with open(json_file_path, 'r') as j:
     params = json.loads(j.read())

In [4]:
params

{'inputs': {'valley_name': 'Deming',
  'TO_DROP': ['1947_09_14',
   '1970_09_09',
   '1974_08_10',
   '1977_09_27',
   '1987_08_21',
   '1990_09_05'],
  'erosion_polygon_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/erosion.geojson',
  'erosion_by_date_polygon_file': None,
  'glacier_polygons_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/glaciers.geojson',
  'dems_path': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems',
  'valley_bounds_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/valley_bounds.geojson',
  'plot_output_dir': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/plots/',
  'uncertainty_file': 'outputs/uncertainty_deming.pcl',
  'reference_dem_date': '2015_09_01',
  'strip_time_format': '%Y_%m_%d',
  'TO_COREGISTER': False,
  'SAVE_DDEMS': True,
  'EROSION_BY_DATE': False,
  'INTERPOLATE': True,
  'FILTER_OUTLIERS': True,
  'SIMPLE_FILTER': True,
  'MASK_GLACIER_SIG

In [5]:
TO_DROP = params['inputs']['TO_DROP']
input_transects_file = params['transects']['input_transects_file']
input_dems_path = params['inputs']['dems_path']
glacier_polygons_file = params['inputs']['glacier_polygons_file']
LINE_COMPLEXITY = params['transects']['line_complexity']
raster_fns = glob.glob(os.path.join(input_dems_path, "*.tif"))

strip_time_format = params['inputs']['strip_time_format']

reference_dem_date = datetime.datetime.strptime(
    params['inputs']['reference_dem_date'], 
    strip_time_format
)

In [6]:

raster_fns = [fn for fn in raster_fns if Path(fn).stem not in TO_DROP]
raster_fns

['/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1970_09_29.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1979_10_06.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1991_09_09.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2013_09_13.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2015_09_01.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2019_10_11.tif']

# Extract profiles from DEMs 

Along each cross-section, extract point with lowest elevation and calculate "path distance", the distance from the furthest downstream cross section line.

In [7]:
# read cross sections file into GeoDataframe
gdf = gpd.read_file(input_transects_file)
# Increase the number of points in each line
gdf.geometry = gdf.geometry.apply(lambda g: profiling_tools.increase_line_complexity(g, LINE_COMPLEXITY))
# Get all points from the cross section lines and create a row for each point. 
gdf['coords'] = gdf.geometry.apply(lambda x: list(x.coords))
crs = gdf.crs
gdf = gpd.GeoDataFrame(pd.DataFrame(gdf).explode('coords', ignore_index=True))
# Make the coords column a shapely.geometry.Point type and drop the cross section geometries which we no longer need.
gdf['coords'] = gdf['coords'].apply(shapely.geometry.Point)
gdf.drop(columns=["geometry"])

combined_gdf = gpd.GeoDataFrame(crs=crs)

for raster in raster_fns:
    print(raster)
    # Extract an elevation value for each point
    with rasterio.open(raster) as src:
        new_gdf = gdf.copy()
        new_gdf['elevation'] = pd.Series([sample[0] for sample in src.sample(new_gdf["coords"].apply(lambda x: (x.xy[0][0], x.xy[1][0])))])
        new_gdf['elevation'] = new_gdf['elevation'].apply(lambda x: np.nan if x == src.nodata else x)

    # Convert file name to datetime as per the provided format
    date = datetime.datetime.strptime(Path(raster).stem, strip_time_format)
    new_gdf['time'] = date

    # Set the geometry to the coords to calculate "path distance"    
    combined_gdf = combined_gdf.append(new_gdf)

combined_gdf

/tmp/ipykernel_72525/3779783694.py:13: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  combined_gdf = gpd.GeoDataFrame(crs=crs)


/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1970_09_29.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1979_10_06.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/1991_09_09.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2013_09_13.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2015_09_01.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/deming/dems/2019_10_11.tif


,id,geometry,coords,elevation,time
0,1,"LINESTRING (583099.089 5399426.143, 583099.146...",POINT (583099.0894011736 5399426.143114396),1154.358154,1970-09-29
1,1,"LINESTRING (583099.089 5399426.143, 583099.146...",POINT (583099.1458772067 5399425.98422849),1154.358154,1970-09-29
2,1,"LINESTRING (583099.089 5399426.143, 583099.146...",POINT (583099.2023532398 5399425.825342583),1154.358154,1970-09-29
3,1,"LINESTRING (583099.089 5399426.143, 583099.146...",POINT (583099.2588292727 5399425.666456678),1154.358154,1970-09-29
4,1,"LINESTRING (583099.089 5399426.143, 583099.146...",POINT (583099.3153053058 5399425.507570772),1154.358154,1970-09-29
...,...,...,...,...,...
2995,6,"LINESTRING (581800.666 5398966.187, 581800.968...",POINT (581952.1226496241 5398857.712493505),1041.504761,2019-10-11
2996,6,"LINESTRING (581800.666 5398966.187, 581800.968...",POINT (581952.4618857836 5398857.542875426),1040.344849,2019-10-11
2997,6,"LINESTRING (581800.666 5398966.187, 581800.968...",POINT (581952.8011219432 5398857.373257346),1041.244385,2019-10-11
2998,6,"LINESTRING (581800.666 5398966.187, 581800.968...",POINT (581953.1403581026 5398857.203639266),1041.244385,2019-10-11


In [8]:
new_gdf = gpd.GeoDataFrame(crs=crs)
for key, group in combined_gdf.groupby(["id", "time"]):
    group.geometry = group['coords']
    group['path_distance'] = pd.Series(group.distance(group.shift(1)).fillna(0)).cumsum()
    new_gdf = new_gdf.append(group)

/tmp/ipykernel_72525/2073592564.py:1: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  new_gdf = gpd.GeoDataFrame(crs=crs)
/tmp/ipykernel_72525/2073592564.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:32610
Right CRS: None

  group['path_distance'] = pd.Series(group.distance(group.shift(1)).fillna(0)).cumsum()


# Mark points as (non)glacial

In [9]:
glaciers_gdf = gpd.read_file(glacier_polygons_file)
glaciers_gdf = glaciers_gdf.to_crs(new_gdf.crs)
glaciers_gdf['time'] = glaciers_gdf['year'].apply(lambda d: datetime.datetime.strptime(d, strip_time_format))

In [10]:
new_gdf['glacial'] = new_gdf.apply(
    lambda row: any(glaciers_gdf.loc[glaciers_gdf['time'] == row["time"], 'geometry'].apply(lambda g: g.contains(row['coords']))),
    axis=1
)

In [11]:
src = new_gdf[[ "time", "path_distance", "elevation", "id", "glacial"]].reset_index()
src['time'] = src['time'].apply(lambda x: x.strftime("%Y-%m-%d"))
alt.Chart(
    src
).transform_filter(
    datum.glacial==False
).mark_line().encode(
    alt.X("path_distance:Q", scale=alt.Scale(zero=False)),
    alt.Y("elevation:Q", scale=alt.Scale(zero=False)),
    alt.Color("time:O", scale=alt.Scale(scheme='turbo')),
).facet(
    row="id:O"
).resolve_scale(
    x="independent",
    y="independent"
).properties(
    # width = 1400,
    # height = 600
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.FacetChart(...)